In [73]:
import moviepy
from moviepy.editor import VideoFileClip
from moviepy.video.io import ImageSequenceClip
from PIL import Image
from IPython.display import HTML

In [1]:
import pandas as pd
import random
import cv2
import os
import matplotlib.pyplot as plt
%matplotlib inline
import time
import numpy as np

In [2]:
#载入追踪数据
df = pd.read_csv('tracker_results.csv')
df.head()

,frame_num,id,bbox_xmin,bbox_ymin,bbox_xmax,bbox_ymax
0,0,-1,261.01556,813.442000,363.86530,1072.62330
1,0,-1,1614.05760,636.934940,1704.69370,837.91077
2,0,-1,883.42316,416.066000,966.37427,594.06540
3,0,-1,1456.64400,47.858955,1525.22530,168.81123
4,0,-1,1654.75800,133.837520,1710.17480,273.05527


In [3]:
df= df[df.frame_num <=600]
#创建词典对违反社交距离次数进行计数
track_list = df.id.unique()
track_violations = { i : [] for i in track_list }

In [4]:
#记录‘安全’与‘违反’
df['safe'] = 0
df['count_violations']=0

创建辅助函数

In [5]:
def distance_boxes(boxA, boxB):
    import math
    center_boxA = [(boxA[0] + boxA[2])/ 2.0, (boxA[1] + boxA[3])/2.0]
    center_boxB = [(boxB[0] + boxB[2])/ 2.0, (boxB[1] + boxB[3])/2.0]
    pixel_distance  = math.sqrt( ((center_boxA[0]-center_boxB[0])**2)+((center_boxA[1]-center_boxB[1])**2) )
    return pixel_distance

In [6]:
def get_bbox(frame_num, track):
    bbox_list = []
    data = df[(df.frame_num == frame_num) & (df.id == track)].reset_index()
    if data.shape[0] > 0:
        x1 = int(data.at[0, 'bbox_xmin'])
        bbox_list.append(x1)
        y1 = int(data.at[0, 'bbox_ymin'])
        bbox_list.append(y1)
        x2 = int(data.at[0, 'bbox_xmax'])
        bbox_list.append(x2)
        y2 = int(data.at[0, 'bbox_ymax'])
        bbox_list.append(y2)        
    return bbox_list

In [7]:
#计算个体之间的距离，高亮违反安全距离的个体
def model_distancing(df,  proximity):

    print("Simulation Started")
    time1 = time.time()
    max_frame = df['frame_num'].max()
    for frame_num in range(3, max_frame):
        pairs_done = []
        if frame_num % 50 == 0:
            print("Completed: ", frame_num)
        
        tracks_this_frame = list(df[df.frame_num==frame_num]['id'])
        
        for track in tracks_this_frame:
            for other in tracks_this_frame:

                if track == other:
                    pass
                else:
                    this_pair = str(track) + "-" + str(other)
                    if this_pair not in pairs_done:
                        pixel_dist = distance_boxes(get_bbox(frame_num, track), get_bbox(frame_num, other))
                        
                        reverse_pair = str(other) + "-" + str(track)                
                        pairs_done.append(this_pair)
                        pairs_done.append(reverse_pair)
                        if pixel_dist < proximity:
                            
                            index_tr = np.where((df.frame_num == frame_num) & (df.id == track))[0].tolist()
                            index_oth = np.where((df.frame_num == frame_num) & (df.id == other))[0].tolist()
                            df.set_value(index_tr[0], 'safe', 1)
                            df.set_value(index_oth[0], 'safe', 1)
                            
                            track_violations[track].append(other)
                            track_violations[other].append(track)
                            
            index_tr = np.where((df.frame_num == frame_num) & (df.id == track))[0].tolist()
            df.set_value(index_tr[0], 'count_violations', len(list(set(track_violations[track]))))
        
                       
    print("Simulation Complete")
    print("Time Taken: ", time.time() - time1)
    return df

In [8]:
proximity = 70

In [9]:
df = model_distancing(df, proximity)

Simulation Started


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


Completed:  50
Completed:  100
Completed:  150
Completed:  200
Completed:  250
Completed:  300
Completed:  350
Completed:  400
Completed:  450
Completed:  500
Completed:  550
Simulation Complete
Time Taken:  442.94570302963257


Analyzing the results

In [72]:
df.head()

,frame_num,id,bbox_xmin,bbox_ymin,bbox_xmax,bbox_ymax,safe,count_violations
0,0,-1,261.01556,813.442000,363.86530,1072.62330,0,0
1,0,-1,1614.05760,636.934940,1704.69370,837.91077,0,0
2,0,-1,883.42316,416.066000,966.37427,594.06540,0,0
3,0,-1,1456.64400,47.858955,1525.22530,168.81123,0,0
4,0,-1,1654.75800,133.837520,1710.17480,273.05527,0,0


In [11]:
df['safe'].value_counts()

0    8599
1    1922
Name: safe, dtype: int64

In [12]:
df['count_violations'].value_counts()

0     4367
1     2344
2     1584
4      743
3      696
5      351
6      322
7       78
10      19
8       17
Name: count_violations, dtype: int64

应用于街头行人视频中

In [14]:
def get_safe(frame_num, track):
    data = df[(df.frame_num == frame_num) & (df.id == track)].reset_index()
    return int(data.at[0, 'safe']), int(data.at[0, 'count_violations'])

In [63]:
if not os.path.exists('./output1'):
    os.mkdir('./output1')

In [77]:
tracks_this_frame = list(df[df.frame_num==frame_num]['id'])

tracks_this_frame

[5, 14, 19, 25, 28, 30, 31, 36, 40, 41, 44, 45, 46, 47, 48, 49, 51, 52]

In [64]:
video_name = 'video/TownCentreXVID_small1.mp4'
cap = cv2.VideoCapture(video_name)
ret = True
count = 0
currentframe=0
ignore_tracks = [10, 14, 27, 20, 35, 42, 26, 16, 28]
max_frame = df['frame_num'].max()

while ret:
    for frame_num in range(3, max_frame):
        ret, image_np = cap.read()
        count += 1
        if count == 2:
            count = 0
            tracks_this_frame = list(df[df.frame_num==frame_num]['id'])
            
            for track in tracks_this_frame:
                 if track in ignore_tracks:
                    pass
                 else:
                    coords = get_bbox(frame_num, track)
                    safe_behavior, count_violations = get_safe(frame_num, track)
                    if safe_behavior == 0:
                        color = (0, 255, 0)
                    else:
                        color = (0, 0, 255)
                
                    text = "Violation: " + str(count_violations)
                    if count_violations == 0:
                        color_text = (0,0,0)
                    else:
                        color_text = (0, 0, 255)
                    cv2.rectangle(image_np, (coords[0], coords[1]) , (coords[2], coords[3]), color,2)
                    cv2.putText(image_np, text, (coords[0]-8,coords[1]-8), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_text, 2, cv2.LINE_AA)
                    if safe_behavior == 1:    
                        cv2.putText(image_np, str("Too Close"), (coords[2]+ 5,coords[3]+ 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2, cv2.LINE_AA)
       
                    file_name = './output1/frame' + str(currentframe) + '.jpg'
                    print ('Creating...' + file_name)
                    cv2.imwrite(file_name, image_np)
                    currentframe += 1


Creating..../output1/frame0.jpg
Creating..../output1/frame1.jpg
Creating..../output1/frame2.jpg
Creating..../output1/frame3.jpg
Creating..../output1/frame4.jpg
Creating..../output1/frame5.jpg
Creating..../output1/frame6.jpg
Creating..../output1/frame7.jpg
Creating..../output1/frame8.jpg
Creating..../output1/frame9.jpg
Creating..../output1/frame10.jpg
Creating..../output1/frame11.jpg
Creating..../output1/frame12.jpg
Creating..../output1/frame13.jpg
Creating..../output1/frame14.jpg
Creating..../output1/frame15.jpg
Creating..../output1/frame16.jpg
Creating..../output1/frame17.jpg
Creating..../output1/frame18.jpg
Creating..../output1/frame19.jpg
Creating..../output1/frame20.jpg
Creating..../output1/frame21.jpg
Creating..../output1/frame22.jpg
Creating..../output1/frame23.jpg
Creating..../output1/frame24.jpg
Creating..../output1/frame25.jpg
Creating..../output1/frame26.jpg
Creating..../output1/frame27.jpg
Creating..../output1/frame28.jpg
Creating..../output1/frame29.jpg
Creating..../output1

Creating..../output1/frame246.jpg
Creating..../output1/frame247.jpg
Creating..../output1/frame248.jpg
Creating..../output1/frame249.jpg
Creating..../output1/frame250.jpg
Creating..../output1/frame251.jpg
Creating..../output1/frame252.jpg
Creating..../output1/frame253.jpg
Creating..../output1/frame254.jpg
Creating..../output1/frame255.jpg
Creating..../output1/frame256.jpg
Creating..../output1/frame257.jpg
Creating..../output1/frame258.jpg
Creating..../output1/frame259.jpg
Creating..../output1/frame260.jpg
Creating..../output1/frame261.jpg
Creating..../output1/frame262.jpg
Creating..../output1/frame263.jpg
Creating..../output1/frame264.jpg
Creating..../output1/frame265.jpg
Creating..../output1/frame266.jpg
Creating..../output1/frame267.jpg
Creating..../output1/frame268.jpg
Creating..../output1/frame269.jpg
Creating..../output1/frame270.jpg
Creating..../output1/frame271.jpg
Creating..../output1/frame272.jpg
Creating..../output1/frame273.jpg
Creating..../output1/frame274.jpg
Creating..../o

Creating..../output1/frame487.jpg
Creating..../output1/frame488.jpg
Creating..../output1/frame489.jpg
Creating..../output1/frame490.jpg
Creating..../output1/frame491.jpg
Creating..../output1/frame492.jpg
Creating..../output1/frame493.jpg
Creating..../output1/frame494.jpg
Creating..../output1/frame495.jpg
Creating..../output1/frame496.jpg
Creating..../output1/frame497.jpg
Creating..../output1/frame498.jpg
Creating..../output1/frame499.jpg
Creating..../output1/frame500.jpg
Creating..../output1/frame501.jpg
Creating..../output1/frame502.jpg
Creating..../output1/frame503.jpg
Creating..../output1/frame504.jpg
Creating..../output1/frame505.jpg
Creating..../output1/frame506.jpg
Creating..../output1/frame507.jpg
Creating..../output1/frame508.jpg
Creating..../output1/frame509.jpg
Creating..../output1/frame510.jpg
Creating..../output1/frame511.jpg
Creating..../output1/frame512.jpg
Creating..../output1/frame513.jpg
Creating..../output1/frame514.jpg
Creating..../output1/frame515.jpg
Creating..../o

Creating..../output1/frame731.jpg
Creating..../output1/frame732.jpg
Creating..../output1/frame733.jpg
Creating..../output1/frame734.jpg
Creating..../output1/frame735.jpg
Creating..../output1/frame736.jpg
Creating..../output1/frame737.jpg
Creating..../output1/frame738.jpg
Creating..../output1/frame739.jpg
Creating..../output1/frame740.jpg
Creating..../output1/frame741.jpg
Creating..../output1/frame742.jpg
Creating..../output1/frame743.jpg
Creating..../output1/frame744.jpg
Creating..../output1/frame745.jpg
Creating..../output1/frame746.jpg
Creating..../output1/frame747.jpg
Creating..../output1/frame748.jpg
Creating..../output1/frame749.jpg
Creating..../output1/frame750.jpg
Creating..../output1/frame751.jpg
Creating..../output1/frame752.jpg
Creating..../output1/frame753.jpg
Creating..../output1/frame754.jpg
Creating..../output1/frame755.jpg
Creating..../output1/frame756.jpg
Creating..../output1/frame757.jpg
Creating..../output1/frame758.jpg
Creating..../output1/frame759.jpg
Creating..../o

Creating..../output1/frame972.jpg
Creating..../output1/frame973.jpg
Creating..../output1/frame974.jpg
Creating..../output1/frame975.jpg
Creating..../output1/frame976.jpg
Creating..../output1/frame977.jpg
Creating..../output1/frame978.jpg
Creating..../output1/frame979.jpg
Creating..../output1/frame980.jpg
Creating..../output1/frame981.jpg
Creating..../output1/frame982.jpg
Creating..../output1/frame983.jpg
Creating..../output1/frame984.jpg
Creating..../output1/frame985.jpg
Creating..../output1/frame986.jpg
Creating..../output1/frame987.jpg
Creating..../output1/frame988.jpg
Creating..../output1/frame989.jpg
Creating..../output1/frame990.jpg
Creating..../output1/frame991.jpg
Creating..../output1/frame992.jpg
Creating..../output1/frame993.jpg
Creating..../output1/frame994.jpg
Creating..../output1/frame995.jpg
Creating..../output1/frame996.jpg
Creating..../output1/frame997.jpg
Creating..../output1/frame998.jpg
Creating..../output1/frame999.jpg
Creating..../output1/frame1000.jpg
Creating..../

...

Creating..../output1/frame1446.jpg
Creating..../output1/frame1447.jpg
Creating..../output1/frame1448.jpg
Creating..../output1/frame1449.jpg
Creating..../output1/frame1450.jpg
Creating..../output1/frame1451.jpg
Creating..../output1/frame1452.jpg
Creating..../output1/frame1453.jpg
Creating..../output1/frame1454.jpg
Creating..../output1/frame1455.jpg
Creating..../output1/frame1456.jpg
Creating..../output1/frame1457.jpg
Creating..../output1/frame1458.jpg
Creating..../output1/frame1459.jpg
Creating..../output1/frame1460.jpg
Creating..../output1/frame1461.jpg
Creating..../output1/frame1462.jpg
Creating..../output1/frame1463.jpg
Creating..../output1/frame1464.jpg
Creating..../output1/frame1465.jpg
Creating..../output1/frame1466.jpg
Creating..../output1/frame1467.jpg
Creating..../output1/frame1468.jpg
Creating..../output1/frame1469.jpg
Creating..../output1/frame1470.jpg
Creating..../output1/frame1471.jpg
Creating..../output1/frame1472.jpg
Creating..../output1/frame1473.jpg
Creating..../output1

Creating..../output1/frame1682.jpg
Creating..../output1/frame1683.jpg
Creating..../output1/frame1684.jpg
Creating..../output1/frame1685.jpg
Creating..../output1/frame1686.jpg
Creating..../output1/frame1687.jpg
Creating..../output1/frame1688.jpg
Creating..../output1/frame1689.jpg
Creating..../output1/frame1690.jpg
Creating..../output1/frame1691.jpg
Creating..../output1/frame1692.jpg
Creating..../output1/frame1693.jpg
Creating..../output1/frame1694.jpg
Creating..../output1/frame1695.jpg
Creating..../output1/frame1696.jpg
Creating..../output1/frame1697.jpg
Creating..../output1/frame1698.jpg
Creating..../output1/frame1699.jpg
Creating..../output1/frame1700.jpg
Creating..../output1/frame1701.jpg
Creating..../output1/frame1702.jpg
Creating..../output1/frame1703.jpg
Creating..../output1/frame1704.jpg
Creating..../output1/frame1705.jpg
Creating..../output1/frame1706.jpg
Creating..../output1/frame1707.jpg
Creating..../output1/frame1708.jpg
Creating..../output1/frame1709.jpg
Creating..../output1

Creating..../output1/frame1918.jpg
Creating..../output1/frame1919.jpg
Creating..../output1/frame1920.jpg
Creating..../output1/frame1921.jpg
Creating..../output1/frame1922.jpg
Creating..../output1/frame1923.jpg
Creating..../output1/frame1924.jpg
Creating..../output1/frame1925.jpg
Creating..../output1/frame1926.jpg
Creating..../output1/frame1927.jpg
Creating..../output1/frame1928.jpg
Creating..../output1/frame1929.jpg
Creating..../output1/frame1930.jpg
Creating..../output1/frame1931.jpg
Creating..../output1/frame1932.jpg
Creating..../output1/frame1933.jpg
Creating..../output1/frame1934.jpg
Creating..../output1/frame1935.jpg
Creating..../output1/frame1936.jpg
Creating..../output1/frame1937.jpg
Creating..../output1/frame1938.jpg
Creating..../output1/frame1939.jpg
Creating..../output1/frame1940.jpg
Creating..../output1/frame1941.jpg
Creating..../output1/frame1942.jpg
Creating..../output1/frame1943.jpg
Creating..../output1/frame1944.jpg
Creating..../output1/frame1945.jpg
Creating..../output1

Creating..../output1/frame2153.jpg
Creating..../output1/frame2154.jpg
Creating..../output1/frame2155.jpg
Creating..../output1/frame2156.jpg
Creating..../output1/frame2157.jpg
Creating..../output1/frame2158.jpg
Creating..../output1/frame2159.jpg
Creating..../output1/frame2160.jpg
Creating..../output1/frame2161.jpg
Creating..../output1/frame2162.jpg
Creating..../output1/frame2163.jpg
Creating..../output1/frame2164.jpg
Creating..../output1/frame2165.jpg
Creating..../output1/frame2166.jpg


error: OpenCV(4.2.0) /Users/travis/build/skvark/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp:715: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'


In [65]:
def delete_dot_ds_store(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            if filename == '.DS_Store':
                os.remove(os.path.join(root, filename))

In [66]:
delete_dot_ds_store('./output1/')

In [67]:
def pics2video(frames_dir, video_dst, name, fps):
    frames_name = os.listdir(frames_dir)
    frames_name = sorted(frames_name,key=lambda x:int(x[5:-4]))   
    frames_path = [frames_dir+frame_name for frame_name in frames_name]
    clip = ImageSequenceClip.ImageSequenceClip(frames_path, fps=fps)
    clip.write_videofile(video_dst+'{}.mp4'.format(name))

In [71]:
pics2video('./output1/','./','video_output',75)
from IPython.display import HTML
HTML("""
<video width="960" height="480" controls>
  <source src="./video_output.mp4" type="video/mp4">
</video>
""")

t:   0%|          | 3/2166 [00:00<01:53, 19.10it/s, now=None]

Moviepy - Building video ./video_output.mp4.
Moviepy - Writing video ./video_output.mp4



Moviepy - Done !
Moviepy - video ready ./video_output.mp4
